In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


The goal is to create one "mastertable" where I have the cities with respective years, then population, crime_count, median_income, unemployment rate, and education for each!

In [ ]:
import pandas as pd

for file in ["crime_chicago.csv", "crime_newyorkcity.csv", "df_median_income.csv", "df_full_interpolated.csv"]:
    df = pd.read_csv(f"/content/drive/MyDrive/Group_Project/data/{file}")
    print(file, df.isna().sum())

crime_chicago.csv year           0
crime_count    0
dtype: int64
crime_newyorkcity.csv year           0
crime_count    0
dtype: int64
df_median_income.csv year                 0
city                 0
median_income_usd    0
dtype: int64
df_full_interpolated.csv city                     0
year                     0
unemployment_rate_pct    0
bachelors_plus_pct       0
dtype: int64


In [ ]:
from pathlib import Path

BASE = Path("/content/drive/MyDrive/Group_Project/data")

print("Files in data dir:")
for p in BASE.glob("*"):
    print(" -", p.name)

Files in data dir:
 - openmeteo_monthly_temp.csv
 - population_per_year.csv
 - avg_educational_score.csv
 - final_df_per_100k.csv
 - osm_per_100k.csv
 - big_merge_crime_with_osm.csv
 - ward42_3_4_5_2012_2022_2023_2024.csv
 - ward_table_2012_2022_2023_2024.csv
 - wiki_summary.csv
 - out.csv
 - osm_surveillance_police_counts.csv
 - crime_chicago.csv
 - crime_newyorkcity.csv
 - df_median_income.csv
 - df_full_interpolated.csv


In [ ]:
paths = {
    "pop": BASE / "out.csv",
    "crime_chi": BASE / "crime_chicago.csv",
    "crime_nyc": BASE / "crime_newyorkcity.csv",
    "osm":BASE / "osm_surveillance_police_counts.csv",
    "income":BASE / "df_median_income.csv",
    "edu":BASE / "df_full_interpolated.csv",
}

pop_both = pd.read_csv(paths["pop"])
crime_chicago = pd.read_csv(paths["crime_chi"]).assign(city="Chicago")
crime_nyc = pd.read_csv(paths["crime_nyc"]).assign(city="New York City")
crime_both = pd.concat([crime_chicago, crime_nyc], ignore_index=True)

osm = pd.read_csv(paths["osm"])
income = pd.read_csv(paths["income"])
edu_unemp = pd.read_csv(paths["edu"])

print(pop_both.shape, crime_both.shape, osm.shape, income.shape, edu_unemp.shape)

(38, 3) (43, 3) (2, 3) (38, 3) (38, 4)


In [ ]:
crime_both

,year,crime_count,city
0,2001,485954,Chicago
1,2002,486830,Chicago
2,2003,475996,Chicago
3,2004,469439,Chicago
4,2005,453785,Chicago
5,2006,448198,Chicago
6,2007,437105,Chicago
7,2008,427211,Chicago
8,2009,392860,Chicago
9,2010,370557,Chicago


--> Filter Chicago crimes for 2006-2024


In [ ]:
crime_both = crime_both.query("2006 <= year <= 2024").copy()

crime_both["year"] = crime_both["year"].astype(int)
crime_both["crime_count"] = crime_both["crime_count"].astype(int)
crime_both["city"] = crime_both["city"].astype(str)

crime_both = crime_both.drop_duplicates(subset=["city","year"], keep="last")

assert crime_both.groupby("city")["year"].nunique().eq(19).all(), \
    f"Missing years:\n{crime_both.groupby('city')['year'].unique()}"

print("crime_both shape:", crime_both.shape)

print(crime_both)


crime_both shape: (38, 3)
    year  crime_count           city
5   2006       448198        Chicago
6   2007       437105        Chicago
7   2008       427211        Chicago
8   2009       392860        Chicago
9   2010       370557        Chicago
10  2011       352040        Chicago
11  2012       336367        Chicago
12  2013       307603        Chicago
13  2014       275869        Chicago
14  2015       264869        Chicago
15  2016       269932        Chicago
16  2017       269265        Chicago
17  2018       269108        Chicago
18  2019       261658        Chicago
19  2020       212646        Chicago
20  2021       209590        Chicago
21  2022       239919        Chicago
22  2023       263137        Chicago
23  2024       258729        Chicago
24  2006       530211  New York City
25  2007       535507  New York City
26  2008       528991  New York City
27  2009       511241  New York City
28  2010       510398  New York City
29  2011       498944  New York City
30  2012    

In [ ]:
final_df = (
    pop_both
    .merge(crime_both, on=["city", "year"], how="left")
    .merge(income, on=["city", "year"], how="left")
    .merge(edu_unemp, on=["city", "year"], how="left")
    .sort_values(["city", "year"])
    .reset_index(drop=True)
)

final_df

,year,city,population,crime_count,median_income_usd,unemployment_rate_pct,bachelors_plus_pct
0,2006,Chicago,2695598,448198,43223,14.10,33.46
1,2007,Chicago,2695598,437105,45505,14.10,33.46
2,2008,Chicago,2695598,427211,46911,14.10,33.46
3,2009,Chicago,2695598,392860,45734,14.10,33.46
4,2010,Chicago,2695598,370557,44776,14.10,33.46
5,2011,Chicago,2696897,352040,43628,14.10,33.46
6,2012,Chicago,2698196,336367,45214,13.66,34.47
7,2013,Chicago,2699494,307603,47099,12.67,35.13
8,2014,Chicago,2700793,275869,48734,10.91,35.96
9,2015,Chicago,2702092,264869,50702,9.50,36.64


Now: construct a city–year panel for Chicago and New York City (2006–2024) combining population, crime, income, unemployment, education, and OpenStreetMap (OSM) infrastructure.

Since we have osm counts as a static snapshot and not a time series, I will merge them once by city, keep the raw counts identical for all years. To make fair comparisons over time and across cities with different sizes, I want to compute per-capita rates using the year-specific population:

So I aks: what should be normalized per 100k for better and accurate comparison across the two cities?
- crime_count --> crime_rate_per_100k
- osm_cctv_count_static --> cctv_per_100k
- osm_police_count_static --> police_per_100k

In [ ]:
final_df_per_100k = final_df.copy()

In [ ]:
# start by getting crime rate per 100k
final_df_per_100k["crime_rate_per_100k"] = (
    final_df_per_100k["crime_count"] / final_df_per_100k["population"] * 100_000
).round(2)

In [ ]:
final_df_per_100k

,year,city,population,crime_count,median_income_usd,unemployment_rate_pct,bachelors_plus_pct,crime_rate_per_100k
0,2006,Chicago,2695598,448198,43223,14.10,33.46,16627.03
1,2007,Chicago,2695598,437105,45505,14.10,33.46,16215.51
2,2008,Chicago,2695598,427211,46911,14.10,33.46,15848.47
3,2009,Chicago,2695598,392860,45734,14.10,33.46,14574.13
4,2010,Chicago,2695598,370557,44776,14.10,33.46,13746.75
5,2011,Chicago,2696897,352040,43628,14.10,33.46,13053.52
6,2012,Chicago,2698196,336367,45214,13.66,34.47,12466.37
7,2013,Chicago,2699494,307603,47099,12.67,35.13,11394.84
8,2014,Chicago,2700793,275869,48734,10.91,35.96,10214.37
9,2015,Chicago,2702092,264869,50702,9.50,36.64,9802.37


In [ ]:
FINAL_DF_PE_100K = Path("/content/drive/MyDrive/Group_Project/data")
FINAL_DF_PE_100K.mkdir(parents=True, exist_ok=True)

final_df_per_100k.to_csv(FINAL_DF_PE_100K/"final_df_per_100k.csv", index=False)

In [ ]:
final_df_per_100k.dtypes

,0
year,int64
city,object
population,int64
crime_count,int64
median_income_usd,int64
unemployment_rate_pct,float64
bachelors_plus_pct,float64
crime_rate_per_100k,float64


In [ ]:
osm = pd.read_csv(BASE / "osm_surveillance_police_counts.csv")
osm

,city,osm_cctv_count,osm_police_count
0,Chicago,1332,103
1,New York City,677,208


In [ ]:
cctv_map = dict(zip(osm["city"], osm["osm_cctv_count"]))
police_map = dict(zip(osm["city"], osm["osm_police_count"]))

osm_per_100k = final_df[["year", "city", "population"]].copy()

osm_per_100k = osm_per_100k.assign(
    cctv_per_100k = (osm_per_100k["city"].map(cctv_map) / osm_per_100k["population"] * 100_000).round(4),
    police_per_100k = (osm_per_100k["city"].map(police_map) / osm_per_100k["population"] * 100_000).round(4)
).sort_values(["city","year"]).reset_index(drop=True)

osm_per_100k

,year,city,population,cctv_per_100k,police_per_100k
0,2006,Chicago,2695598,49.4139,3.8210
1,2007,Chicago,2695598,49.4139,3.8210
2,2008,Chicago,2695598,49.4139,3.8210
3,2009,Chicago,2695598,49.4139,3.8210
4,2010,Chicago,2695598,49.4139,3.8210
5,2011,Chicago,2696897,49.3901,3.8192
6,2012,Chicago,2698196,49.3663,3.8174
7,2013,Chicago,2699494,49.3426,3.8155
8,2014,Chicago,2700793,49.3188,3.8137
9,2015,Chicago,2702092,49.2951,3.8119


In [ ]:
APPROX_CCTV_PER_100K = Path("/content/drive/MyDrive/Group_Project/data")
APPROX_CCTV_PER_100K.mkdir(parents=True, exist_ok=True)

osm_per_100k.to_csv(APPROX_CCTV_PER_100K/"osm_per_100k.csv", index=False)

In [ ]:
BASE = "/content/drive/MyDrive/Group_Project/data"
crime = pd.read_csv(f"{BASE}/final_df_per_100k.csv")
osm   = pd.read_csv(f"{BASE}/osm_per_100k.csv")

crime_cols = [
    "city","year","population","crime_count","crime_rate_per_100k",
    "median_income_usd","unemployment_rate_pct","bachelors_plus_pct"
]
osm_cols = ["city","year","cctv_per_100k","police_per_100k"]

crime = crime[crime_cols].copy()
osm = osm[osm_cols].copy()

crime["city"] = crime["city"].str.strip()
osm["city"] = osm["city"].str.strip()
crime["year"] = crime["year"].astype(int)
osm["year"] = osm["year"].astype(int)

years = list(range(2006, 2025))
mask_cities = crime["city"].isin(["Chicago","New York City"])
crime = crime[mask_cities & crime["year"].isin(years)]
osm  = osm[osm["city"].isin(["Chicago","New York City"]) & osm["year"].isin(years)]

assert not crime.duplicated(["city","year"]).any(), "crime has duplicate city-year rows"
assert not osm.duplicated(["city","year"]).any(), "osm has duplicate city-year rows"

big_merge_crime = crime.merge(
    osm, on=["city","year"], how="left", validate="one_to_one", indicator=True
)

assert (big_merge_crime["_merge"] == "both").all()
big_merge_crime = big_merge_crime.drop(columns="_merge")
big_merge_crime["Chicago"] = (big_merge_crime["city"] == "Chicago").astype(int)
big_merge_crime = big_merge_crime.sort_values(["city","year"]).reset_index(drop=True)

big_merge_crime_path = f"{BASE}/big_merge_crime_with_osm.csv"
big_merge_crime.to_csv(big_merge_crime_path, index=False)
print("Saved model-ready big merge crime with osm data →", big_merge_crime_path)

display(big_merge_crime)

Saved model-ready big merge crime with osm data → /content/drive/MyDrive/Group_Project/data/big_merge_crime_with_osm.csv


,city,year,population,crime_count,crime_rate_per_100k,median_income_usd,unemployment_rate_pct,bachelors_plus_pct,cctv_per_100k,police_per_100k,Chicago
0,Chicago,2006,2695598,448198,16627.03,43223,14.10,33.46,49.4139,3.8210,1
1,Chicago,2007,2695598,437105,16215.51,45505,14.10,33.46,49.4139,3.8210,1
2,Chicago,2008,2695598,427211,15848.47,46911,14.10,33.46,49.4139,3.8210,1
3,Chicago,2009,2695598,392860,14574.13,45734,14.10,33.46,49.4139,3.8210,1
4,Chicago,2010,2695598,370557,13746.75,44776,14.10,33.46,49.4139,3.8210,1
5,Chicago,2011,2696897,352040,13053.52,43628,14.10,33.46,49.3901,3.8192,1
6,Chicago,2012,2698196,336367,12466.37,45214,13.66,34.47,49.3663,3.8174,1
7,Chicago,2013,2699494,307603,11394.84,47099,12.67,35.13,49.3426,3.8155,1
8,Chicago,2014,2700793,275869,10214.37,48734,10.91,35.96,49.3188,3.8137,1
9,Chicago,2015,2702092,264869,9802.37,50702,9.50,36.64,49.2951,3.8119,1


Finally I compare Chicago wards. 42 is downtown Chicago (one of the most heavily surveilled areas) while 3, 4, and 5 are areas that saw recent policy changes in CCTV cameras. We aim to compare their crime_count, obviously standardizing to crime_count per 1000 for a fair comparison.

In [ ]:
# Wards 42, 3, 4, 5 for the years: 2012, 2022, 2023, 2024
# group-count from Socrata + per-1,000 rates + table + plot

import requests, pandas as pd
import matplotlib.pyplot as plt
import re

BASE = "/content/drive/MyDrive/Group_Project/data"
SOCRATA = "https://data.cityofchicago.org/resource/ijzp-q8t2.json"

years = [2012, 2022, 2023, 2024]
wards = [42, 3, 4, 5]

POP = {
    (42, 2012): 66000,
    (42, 2022): 72000,
    (42, 2023): 72572,
    (42, 2024): 73000,

    (3,  2012): 50000,
    (3,  2022): 55000,
    (3,  2023): 55444,
    (3,  2024): 57000,

    (4,  2012): 52000,
    (4,  2022): 52500,
    (4,  2023): 51937,
    (4,  2024): 52500,

    (5,  2012): 52000,
    (5,  2022): 48500,
    (5,  2023): 47372,
    (5,  2024): 47500,
}

params = {
    "$select": "ward, year, count(1) as n",
    "$where": f"ward in ({','.join(map(str,wards))}) AND year in ({','.join(map(str,years))})",
    "$group": "ward, year",
    "$order": "ward, year"
}
r = requests.get(SOCRATA, params=params)
r.raise_for_status()
df = pd.DataFrame(r.json())
if df.empty:
    raise ValueError("API returned no rows — check filters or dataset availability.")

df["ward"] = df["ward"].astype(int)
df["year"] = df["year"].astype(int)
df["crime_count"] = df["n"].astype(int)
df.drop(columns=["n"], inplace=True)

df["population"] = [POP.get((w, y)) for w, y in zip(df["ward"], df["year"])]

if df["population"].isna().any():
    missing_wards_years = df[df["population"].isna()][["ward", "year"]].values.tolist()
    raise ValueError(f"Missing population data for ward-year combinations: {missing_wards_years}")

df["crime_per_1000"] = df["crime_count"] / df["population"] * 1000
df["crime_per_1000"] = df["crime_per_1000"].round(2)

df = df.sort_values(["ward","year"])
out_path = f"{BASE}/ward42_3_4_5_2012_2022_2023_2024.csv"
df.to_csv(out_path, index=False)
print("Saved →", out_path)
display(df)

Saved → /content/drive/MyDrive/Group_Project/data/ward42_3_4_5_2012_2022_2023_2024.csv


,ward,year,crime_count,population,crime_per_1000
0,3,2012,8628,50000,172.56
1,3,2022,7470,55000,135.82
2,3,2023,7309,55444,131.83
3,3,2024,6812,57000,119.51
4,4,2012,5396,52000,103.77
5,4,2022,7846,52500,149.45
6,4,2023,8448,51937,162.66
7,4,2024,8279,52500,157.70
8,5,2012,8429,52000,162.10
9,5,2022,6167,48500,127.15


I have now built final_df_per_100k with columns
["year","city","population","crime_count","crime_rate_per_100k","median_income_usd","unemployment_rate_pct","bachelors_plus_pct"]
And I have already built osm_per_100k with columns
["year","city","population","cctv_per_100k","police_per_100k"]

In [ ]:
def summarize_by_city(df, cols):
    return (df
            .groupby("city")[cols]
            .agg(["count","mean","median","min","max"])
            .round(2))

very_much_needed_stats = summarize_by_city(final_df_per_100k,
                               ["crime_rate_per_100k",
                                "median_income_usd","unemployment_rate_pct","bachelors_plus_pct"])

display(very_much_needed_stats)

crime_rate_per_100k                                        \
                            count      mean   median      min       max   
city                                                                      
Chicago                        19  11444.31  9984.94  7772.49  16627.03   
New York City                  19   5943.85  6056.59  4703.15   6689.78   

              median_income_usd                                   \
                          count      mean   median    min    max   
city                                                               
Chicago                      19  54895.89  50702.0  43223  80613   
New York City                19  59305.95  55752.0  46480  81228   

              unemployment_rate_pct                             \
                              count   mean median   min    max   
city                                                             
Chicago                          19  10.56  10.10  6.29  14.10   
New York City                    19   8.72   8.26  5.15  11.77   

              bachelors_plus_pct                              
                           count   mean median    min    max  
city                                                          
Chicago                       19  37.97  36.64  33.46  46.39  
New York City                 19  37.20  36.84  34.11  42.53

In [ ]:
stats_osm = summarize_by_city(osm_per_100k,
                              ["cctv_per_100k","police_per_100k"])

display(stats_osm)

cctv_per_100k                             police_per_100k        \
                      count   mean median    min    max           count  mean   
city                                                                            
Chicago                  19  49.36  49.40  49.22  49.44              19  3.82   
New York City            19   8.06   8.09   7.69   8.28              19  2.48   

                                  
              median   min   max  
city                              
Chicago         3.82  3.81  3.82  
New York City   2.49  2.36  2.54

In [ ]:
print("Descriptive stats (2006–2024)")
display(very_much_needed_stats)
display(stats_osm)

Descriptive stats (2006–2024)


crime_rate_per_100k                                        \
                            count      mean   median      min       max   
city                                                                      
Chicago                        19  11444.31  9984.94  7772.49  16627.03   
New York City                  19   5943.85  6056.59  4703.15   6689.78   

              median_income_usd                                   \
                          count      mean   median    min    max   
city                                                               
Chicago                      19  54895.89  50702.0  43223  80613   
New York City                19  59305.95  55752.0  46480  81228   

              unemployment_rate_pct                             \
                              count   mean median   min    max   
city                                                             
Chicago                          19  10.56  10.10  6.29  14.10   
New York City                    19   8.72   8.26  5.15  11.77   

              bachelors_plus_pct                              
                           count   mean median    min    max  
city                                                          
Chicago                       19  37.97  36.64  33.46  46.39  
New York City                 19  37.20  36.84  34.11  42.53

cctv_per_100k                             police_per_100k        \
                      count   mean median    min    max           count  mean   
city                                                                            
Chicago                  19  49.36  49.40  49.22  49.44              19  3.82   
New York City            19   8.06   8.09   7.69   8.28              19  2.48   

                                  
              median   min   max  
city                              
Chicago         3.82  3.81  3.82  
New York City   2.49  2.36  2.54

In [ ]:
import pandas as pd
import numpy as np

df = final_df_per_100k.copy()

base_end = (
    df.query("year in [2006, 2024]")
      .pivot_table(index="city", columns="year",
                   values=["crime_count","crime_rate_per_100k"])
)

base_end.columns = [f"{v}_{y}" for v, y in base_end.columns]
base_end = base_end.reset_index()

base_end["crime_rate_abs_change"] = (base_end["crime_rate_per_100k_2024"] - base_end["crime_rate_per_100k_2006"]).round(2)
base_end["crime_rate_pct_change"] = ((base_end["crime_rate_per_100k_2024"]/base_end["crime_rate_per_100k_2006"] - 1)*100).round(2)

base_end["crime_count_abs_change"] = (base_end["crime_count_2024"] - base_end["crime_count_2006"]).astype(int)
base_end["crime_count_pct_change"] = ((base_end["crime_count_2024"]/base_end["crime_count_2006"] - 1)*100).round(2)

headline = base_end.loc[:, ["city",
                            "crime_rate_per_100k_2006","crime_rate_per_100k_2024","crime_rate_pct_change",
                            "crime_count_2006","crime_count_2024","crime_count_abs_change","crime_count_pct_change"]]

print("💡Let's look at most important numbers (2006 → 2024) changes💡")
for _, r in headline.iterrows():
    print(
        f"{r['city']}: crime rate per 100k {r['crime_rate_per_100k_2006']} → {r['crime_rate_per_100k_2024']} "
        f"({r['crime_rate_pct_change']}%),  total crime {int(r['crime_count_2006'])} → {int(r['crime_count_2024'])} "
        f"({int(r['crime_count_abs_change'])}, {r['crime_count_pct_change']}%)."
    )

display(headline)


💡Let's look at most important numbers (2006 → 2024) changes💡
Chicago: crime rate per 100k 16627.03 → 9594.78 (-42.29%),  total crime 448198 → 258729 (-189469, -42.27%).
New York City: crime rate per 100k 6485.66 → 6665.7 (2.78%),  total crime 530211 → 565118 (34907, 6.58%).


,city,crime_rate_per_100k_2006,crime_rate_per_100k_2024,crime_rate_pct_change,crime_count_2006,crime_count_2024,crime_count_abs_change,crime_count_pct_change
0,Chicago,16627.03,9594.78,-42.29,448198.0,258729.0,-189469,-42.27
1,New York City,6485.66,6665.70,2.78,530211.0,565118.0,34907,6.58


In [ ]:
pct_summary = headline[["city","crime_rate_pct_change","crime_count_pct_change"]].copy()
pct_summary = pct_summary.rename(columns={
    "crime_rate_pct_change": "crime_rate_pct_change_2006_2024",
    "crime_count_pct_change": "crime_count_pct_change_2006_2024",
})
display(pct_summary)


,city,crime_rate_pct_change_2006_2024,crime_count_pct_change_2006_2024
0,Chicago,-42.29,-42.27
1,New York City,2.78,6.58


In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np

BASE = Path("/content/drive/MyDrive/Group_Project/data")
IN_FILE  = BASE / "ward42_3_4_5_2012_2022_2023_2024.csv"
OUT_FILE = BASE / "ward_table_2012_2022_2023_2024.csv"

years = [2012, 2022, 2023, 2024]
wards = [42, 3, 4, 5]

df = pd.read_csv(IN_FILE)

df["ward"] = df["ward"].astype(int)
df["year"] = df["year"].astype(int)
df["crime_per_1000"] = df["crime_per_1000"].astype(float)

tbl = (
    df.pivot(index="ward", columns="year", values="crime_per_1000")
      .reindex(index=wards, columns=years)
)

earliest_year = min(years)
latest_year = max(years)
tbl[f"pct_change_{earliest_year}→{latest_year}"] = (
    (tbl[latest_year] - tbl[earliest_year]) / tbl[earliest_year] * 100
)

display(tbl.round(2))
tbl.round(2).to_csv(OUT_FILE)
print("Saved table →", OUT_FILE)

year,2012,2022,2023,2024,pct_change_2012→2024
ward,,,,,
42,209.56,176.32,133.76,120.63,-42.44
3,172.56,135.82,131.83,119.51,-30.74
4,103.77,149.45,162.66,157.70,51.97
5,162.10,127.15,133.88,123.94,-23.54


Saved table → /content/drive/MyDrive/Group_Project/data/ward_table_2012_2022_2023_2024.csv
